In [1]:
!pip install --user "ibm-watsonx-ai==0.2.6"
!pip install --user "langchain==0.1.16"
!pip install --user "langchain-ibm==0.1.4"
!pip install --user "huggingface == 0.0.1"
!pip install --user "huggingface-hub == 0.23.4"
!pip install --user "sentence-transformers == 2.5.1"
!pip install --user "chromadb"
!pip install --user "wget == 3.2"

  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.23.4 which is incompatible.
transformers 4.47.1 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.23.4 which is incompatible.
  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.28.1-py3-none-any.whl (464 kB)
  Attempting uninstall: hug

# Import required libraries

In [3]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget

from google.colab import userdata
userdata.get('IBM_WATSONX_API_KEY')

ModuleNotFoundError: No module named 'langchain'

# Preprocessing
### Load the documents
this is the `load` step in `Indexing`

In [4]:
file_name = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

wget.download(url, out=file_name)
print('file downloaded')

file downloaded


In [5]:
with open(file_name, 'r') as file:
  contents = file.read()
  print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

### Splitting the document into chunks
the `split` process in `Indexing`

In [6]:
loader = TextLoader(file_name)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

16


the document has been split into 16 chunks

### Embedding and storing
this step is the `embed` and `store` processes in `Indexing`

converting the text into numbers (embedding)

creates a default embedding model from Hugging Face and ingests them to Chromadb

In [7]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
print('document ingested')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

document ingested


# LLM model construction

In [8]:
model_id = 'google/flan-ul2'

In [9]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130,
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.5
}

In [22]:
credentials = {
    "url": "https://jp-tok.ml.cloud.ibm.com",
    "apikey": userdata.get('IBM_WATSONX_API_KEY')
}

project_id = userdata.get('IBM_WATSONX_PROJECT_ID')

print(userdata.get('IBM_WATSONX_API_KEY'))
print(userdata.get('IBM_WATSONX_PROJECT_ID'))

aeJtU1jXxryfKkjANZ1zS18J5mnPouvR8WfC5LZdCNTo
97f68e0f-14f7-47c9-8a63-81bad015dea2


In [23]:
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

Build a model called `flan_ul2_llm` from watsonx.ai

In [24]:
flan_ul2_llm = WatsonxLLM(model=model)

# Integrating LangChain
create a simple Q&A application over the document source using LangChain's `RetrievalQA`

In [25]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "what is the mobile policy?"
qa.invoke(query)

{'query': 'what is the mobile policy?',
 'result': 'The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance. Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations. Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device. Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be discreet when discussing company matters in public spaces. Cost Management: Keep personal phone usage separate from company accounts and reimburse th

In [26]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "can you summarize the document for me?"
qa.invoke(query)

{'query': 'can you summarize the document for me?',
 'result': "Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability. Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest. Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy. Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices

Try to use another model, `LLAMA_3_70B_INSTRUCT`

In [27]:
model_id = 'meta-llama/llama-3-70b-instruct'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.5
}

credentials = {
    "url": "https://jp-tok.ml.cloud.ibm.com",
    "apikey": userdata.get('IBM_WATSONX_API_KEY')
}

project_id = userdata.get('IBM_WATSONX_PROJECT_ID')

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

llama_3_llm = WatsonxLLM(model=model)

In [28]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm,
                                 chain_type='stuff',
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

{'query': 'Can you summarize the document for me?',
 'result': " This document appears to be a collection of policies and codes of conduct for an organization. It outlines the organization's commitment to integrity, respect, accountability, safety, and environmental responsibility. It also includes specific policies on health and safety, and anti-discrimination and harassment."}

### Using prompt template

In [29]:
prompt_template = """Use the information from the document to answer the question at the end. If you do not know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [30]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm,
                                 chain_type='stuff',
                                 retriever=docsearch.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=False)
query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': 'Answer: The document does not mention eating in company vehicles, it only mentions that smoking is not permitted in company vehicles.'}

### Make the conversation have memory

In [31]:
query = "What I cannot do in it?"
qa.invoke(query)

{'query': 'What I cannot do in it?',
 'result': 'Answer: You cannot share your login credentials, download apps or click links from unfamiliar sources, transmit sensitive company information via unsecured messaging apps or emails, and distribute offensive or inappropriate content.'}

In [32]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [34]:
qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm,
                                           chain_type='stuff',
                                           retriever=docsearch.as_retriever(),
                                           memory=memory,
                                           get_chat_history=lambda h : h,
                                           return_source_documents=False
                                           )

In [35]:
history = []

In [36]:
query = "What is mobile policy"
result = qa.invoke({"question": query, "chat_history": history})
print(result['answer'])

 The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization.


In [37]:
history.append((query, result['answer']))

In [38]:
query = "List points in it?"
result = qa.invoke({"question": query, "chat_history": history})
print(result['answer'])

 rephrased question.


In [39]:
history.append((query, result['answer']))

In [40]:
query = "What is the aim of it?"
result = qa.invoke({"question": query, "chat_history": history})
print(result['answer'])

 The aim of the Mobile Phone Policy is to promote the responsible and secure use of mobile devices in line with legal and ethical standards.


# Wrap up and make it an agent

In [42]:
def qa():
  memory = ConversationBufferMemory(memory_key="chat_history", return_message = True)
  qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm,
                                           chain_type='stuff',
                                           retriever=docsearch.as_retriever(),
                                             memory=memory,
                                             get_chat_history=lambda h : h,
                                             return_source_documents=False)
  history = []
  while True:
    query = input("Question: ")
    if (query.lower() in ["quit", "exit", "bye"]) :
      print("Answer: Goodbye!")
      break
    result = qa.invoke({"question": query, "chat_history": history})
    history.append((query, result['answer']))
    print("Answer: ", result['answer'])

In [43]:
qa()

Question: hi there
Answer:   I don't know.
Question: can you help me summarize that document
Answer:    I'd be happy to help you summarize a document. Which document would you like me to summarize?
Question: the policy
Answer:    The policy being referred to is the Health and Safety Policy. It prioritizes the well-being of employees, customers, and the public, and aims to maintain a workplace free from hazards, preventing accidents, injuries, and illnesses.
Question: what is the smoking policy?
Answer:    The smoking policy is that smoking is only permitted in designated smoking areas, marked by appropriate signage, and is strictly prohibited inside company buildings, offices, meeting rooms, and other enclosed spaces, including electronic cigarettes and vaping devices.
Question: can you list all points of it?
Answer:   The specific rules and regulations of the company's smoking policy are as follows: 
- Smoking is only permitted in designated smoking areas, as marked by appropriate sig